In [8]:
import os
import sys

# Set working directory to your project root (where config, src, etc. are)
os.chdir("/home/divyansh-rawat/AIML in Production-Texti")

# Add src to sys.path so Python can find 'textsummarizer'
sys.path.append(os.path.abspath("src"))

In [4]:
import os


In [ ]:
#os.chdir("../")

In [1]:
%pwd

'/home/divyansh-rawat/AIML in Production-Texti/research'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path 
    tokenizer_name: Path

In [15]:
pip install python-box

  Using cached python_box-7.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.0 kB)
Using cached python_box-7.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
from textsummarizer.constants import *  
from textsummarizer.utils.common import read_yaml,create_directories
from ensure import ensure_annotations

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            tokenizer_name=Path(config.tokenizer_name)
        )

        return data_transformation_config

In [ ]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [22]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(
            example_batch['dialogue'], max_length=1024, truncation=True
        )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                example_batch['summary'], max_length=128, truncation=True
            )

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_features, batched=True
        )
        dataset_samsum_pt.save_to_disk(
            os.path.join(self.config.root_dir, "samsum_dataset")
        )

In [27]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-06-24 17:12:40,249 : INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-06-24 17:12:40,250 : INFO: common: YAML file: params.yaml loaded successfully]
[2025-06-24 17:12:40,250 : INFO: common: Created directory at: artifacts]


BoxKeyError: "'Box' object has no attribute 'data_transformation'"